<a href="https://colab.research.google.com/github/Mastemace/GZAPV_2021/blob/main/P20132/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import cv2
import pandas
import glob
import math
from tqdm import tqdm
from scipy.signal import triang

In [74]:
!rm -rf /content/train
!unzip /content/drive/MyDrive/GZAPV/sample.zip -d /content

Archive:  /content/drive/MyDrive/GZAPV/sample.zip
replace /content/sample/10_left.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/sample/10_left.jpeg  
  inflating: /content/sample/10_right.jpeg  
  inflating: /content/sample/13_left.jpeg  
  inflating: /content/sample/13_right.jpeg  
  inflating: /content/sample/15_left.jpeg  
  inflating: /content/sample/15_right.jpeg  
  inflating: /content/sample/16_left.jpeg  
  inflating: /content/sample/16_right.jpeg  
  inflating: /content/sample/17_left.jpeg  
  inflating: /content/sample/17_right.jpeg  


In [26]:
train_img_paths = glob.glob('sample/*.jpeg')
train_img_names = [img_path.split('/')[-1].split('.')[0] for img_path in train_img_paths]
train_data = zip(train_img_paths, train_img_names)

In [27]:
df = pandas.read_csv('/content/drive/MyDrive/GZAPV/trainLabels.csv')
data_dict = {}
for train_img_path ,train_img_name in train_data:
  row = df.loc[df['image'] == train_img_name]
  label = row.iloc[0]['level']
  data_dict[train_img_path] = label

print(data_dict)

{'sample/15_right.jpeg': 2, 'sample/13_left.jpeg': 0, 'sample/15_left.jpeg': 1, 'sample/16_left.jpeg': 4, 'sample/13_right.jpeg': 0, 'sample/10_left.jpeg': 0, 'sample/17_right.jpeg': 1, 'sample/17_left.jpeg': 0, 'sample/16_right.jpeg': 4, 'sample/10_right.jpeg': 0}


In [46]:
def separation_folder_hough(paths, out_dir):
    for path in paths:
        base_name = path.split('/')[1]
        img  = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 10, 10, minRadius = 900, maxRadius = 1700)
        circle  = circles[0][0]
        circle[2] *= 1.05
        
        min_x = np.clip(int(circle[0]-circle[2]), 0, img.shape[1]-1)
        max_x = np.clip(int(circle[0]+circle[2]), 0, img.shape[1]-1)
        min_y = np.clip(int(circle[1]-circle[2]), 0, img.shape[0]-1)
        max_y = np.clip(int(circle[1]+circle[2]), 0, img.shape[0]-1)
        
        img = img[min_y:max_y, min_x:max_x, ...]
        cv2.imwrite(out_dir+'/'+base_name, img)    
        
        
def resize_folder(paths, out_dir, out_res):
    for path in paths:
        base_name = path.split('/')[1]
        img  = cv2.imread(path)
       
        #preskalovani, aby vetsi strana byla rovna out_res
        if img.shape[0]>img.shape[1]:
            dx = int(out_res/img.shape[0] * img.shape[1])
            img = cv2.resize(img, (dx, out_res), interpolation = cv2.INTER_LANCZOS4)
        else:
            dy = int(out_res/img.shape[1] * img.shape[0])
            img = cv2.resize(img, (out_res, dy), interpolation = cv2.INTER_LANCZOS4)
            
        #vycentrovani    
        sy = (out_res-img.shape[0])//2
        sx = (out_res-img.shape[1])//2

        #vlozeni do ctvercoveho obrazku
        out_img = np.zeros((out_res, out_res, 3))
        out_img[sy:img.shape[0]+sy, sx:img.shape[1]+sx, ...] = img
        
        cv2.imwrite(out_dir+'/'+base_name, out_img)
def equalize(paths, out_dir):
  for path in paths:
    base_name = path.split('/')[1]
    img  = cv2.imread(path)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB);
    clahe = cv2.createCLAHE(clipLimit = 1.0, tileGridSize = (8,8))
    img[...,0] = clahe.apply(img[...,0])
    img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    cv2.imwrite(out_dir+'/'+base_name, img)
  

In [75]:
# TODO - studenti pouziji houghovu transformaci z githubu od Petra na oriznuti
separation_folder_hough(train_img_paths, 'sample')
resize_folder(train_img_paths, 'sample', 1024)
equalize(train_img_paths, 'sample')             #opakovaným  spuštěním se mění kvalita obrázku

avg_res  = [1024, 1024]
print(avg_res)

[1024, 1024]


In [62]:
factor = 1
len(data_dict.keys()) * (avg_res[0] / factor) * (avg_res[1] / factor) * 1 * 32 / 8 / 1024 / 1024 / 1024

0.0390625

In [72]:
train_x = np.zeros((len(data_dict.keys()), math.ceil(avg_res[0] / factor) * math.ceil(avg_res[1] / factor)), dtype=np.float32)
train_y = np.zeros(len(data_dict.keys()), dtype=np.int32)

In [64]:
# vyzkouset jiny konvolucni kernel
tr = triang(factor*2+1).reshape(factor*2+1, 1)
kernel = np.dot(tr, tr.T)
kernel /= np.sum(kernel)

In [65]:
i = 0
for img_path in tqdm(data_dict.keys()):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (avg_res[1], avg_res[0]))
  img = cv2.filter2D(img, -1, kernel)[::factor, ::factor, ...]
  # feature extraction, student muze vlozit vlastni vektor features
  train_x[i, ...] = img.flatten() / 255.0
  train_y[i] = data_dict[img_path]
  i += 1

train_y[train_y != 0] = -1
train_y[train_y == 0] = 1

100%|██████████| 10/10 [00:00<00:00, 39.63it/s]


In [66]:
# https://docs.opencv.org/3.4/d1/d73/tutorial_introduction_to_svm.html
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

In [67]:
svm.train(train_x, cv2.ml.ROW_SAMPLE, train_y)

True

In [70]:
# predikovat ne trenovaci data, ale testovaci
predictions = np.copy(train_y)
for i in tqdm(range(train_x.shape[0])):
  predictions[i] = svm.predict(train_x[i:i+1, ...])[1]

ind = predictions == train_y
print(np.sum(ind), np.sum(ind) / train_y.shape[0] * 100)

100%|██████████| 10/10 [00:00<00:00, 1034.33it/s]

10 100.0
